In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.0.3/spark-3.0.3-bin-hadoop2.7.tgz
!tar xf spark-3.0.3-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:14 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:15 http://ppa

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-12-17 02:11:50--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.09s   

2021-12-17 02:11:50 (11.2 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [4]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Vine_Review_Analysis").getOrCreate()


In [5]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Software_v1_00.tsv.gz" 
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42605767|R3EFW2STIYIY0I|B00MUTIDKI|     248732228|McAfee 2015 Inter...|        Software|          1|            2|          2|   N|                Y|I was very disapp...|I was very disapp...| 2015-08-31|
|         US|   51771800|R12NR0R5A9F7FT|B00EPACNUG|     531462352|Hallmark Card Stu...|        Software|          5|    

In [6]:
dropna_df = df.dropna()

In [7]:
vine_df = dropna_df.select(['review_id', 'star_rating','helpful_votes','total_votes','vine','verified_purchase']).drop_duplicates()
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2F7DR75PS8NKT|          5|            0|          0|   N|                Y|
|R3F8SH5XNMIYIH|          4|            0|          0|   N|                Y|
| R5L0WP3MZVTM8|          4|            0|          0|   N|                Y|
|R3UWNSUI0S5571|          1|            1|          2|   N|                Y|
| RD4048QLYIB8Y|          5|            1|          2|   N|                N|
|R138WLFRO0RAZS|          5|            0|          0|   N|                Y|
|R3LTBVRHLYAJWT|          5|            0|          0|   N|                Y|
|R1YAQ9K1ARUYY3|          5|            1|          1|   N|                Y|
|R123ZJH7X500FY|          3|            1|          2|   N|                Y|
|R3TUJRPUVB38KJ|          5|            0|          0|   N|     

In [8]:
# filter for only columns with 20 or more total votes 
cleaned_vine_df = vine_df.filter('total_votes>=20')
cleaned_vine_df.show(truncate=False)

+--------------+-----------+-------------+-----------+----+-----------------+
|review_id     |star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1PXUULGJ2ARI8|3          |20           |20         |N   |Y                |
|RNPOC061DM6SO |1          |90           |96         |N   |N                |
|R23Z76M6ND53B4|1          |32           |36         |N   |Y                |
|R1ZUH17F66XJUY|1          |26           |26         |N   |N                |
|R34DIXDQ7DHDVP|1          |28           |29         |N   |N                |
|R3398IB5T7BMCJ|1          |59           |66         |N   |N                |
|R1E44NTT2LX0F8|1          |5            |23         |N   |Y                |
|R378MR60AUHX09|2          |19           |21         |N   |Y                |
|R1EUBPUEIXIDP6|3          |44           |51         |N   |Y                |
|R137E1FON7IRZS|1          |18           |22         |N   |N    

In [9]:
# filter for only columns with mostly helpful votes
helpful_vine_df = cleaned_vine_df.filter("helpful_votes/total_votes>=0.5")
helpful_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1PXUULGJ2ARI8|          3|           20|         20|   N|                Y|
| RNPOC061DM6SO|          1|           90|         96|   N|                N|
|R23Z76M6ND53B4|          1|           32|         36|   N|                Y|
|R1ZUH17F66XJUY|          1|           26|         26|   N|                N|
|R34DIXDQ7DHDVP|          1|           28|         29|   N|                N|
|R3398IB5T7BMCJ|          1|           59|         66|   N|                N|
|R378MR60AUHX09|          2|           19|         21|   N|                Y|
|R1EUBPUEIXIDP6|          3|           44|         51|   N|                Y|
|R137E1FON7IRZS|          1|           18|         22|   N|                N|
| RE1E7SYQDI27W|          1|           25|         35|   N|     

In [10]:
# load in a sql function to use columns 
from pyspark.sql.functions import col

# Filter last DF to only columns with vine == "Y" (Paid)
helpful_vine_y_df = helpful_vine_df.filter(col("vine") == "Y")
helpful_vine_y_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RW5VVLEG9IFIE|          4|          405|        417|   Y|                N|
|R19S7KGPJBQIH5|          4|           31|         35|   Y|                N|
|R1TOMANDNY482H|          5|           24|         25|   Y|                N|
|R2GSZW8LSS382H|          5|           17|         20|   Y|                N|
|R1LP86XHXKHB6T|          4|           39|         40|   Y|                N|
|R3PIUKLIZC0K2F|          1|           69|         80|   Y|                N|
| RSOU9400W0W5G|          3|           15|         23|   Y|                N|
|R2WNA0FQAHFXKJ|          3|           61|         63|   Y|                N|
|R3TYODQJ379ONQ|          3|           20|         20|   Y|                N|
| RH9EI8TTHE45W|          5|           28|         31|   Y|     

In [11]:
# Filter last DF to only columns with vine == "N" (Unpaid)
helpful_vine_n_df = helpful_vine_df.filter(col("vine") == "N")
helpful_vine_n_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1PXUULGJ2ARI8|          3|           20|         20|   N|                Y|
| RNPOC061DM6SO|          1|           90|         96|   N|                N|
|R23Z76M6ND53B4|          1|           32|         36|   N|                Y|
|R1ZUH17F66XJUY|          1|           26|         26|   N|                N|
|R34DIXDQ7DHDVP|          1|           28|         29|   N|                N|
|R3398IB5T7BMCJ|          1|           59|         66|   N|                N|
|R378MR60AUHX09|          2|           19|         21|   N|                Y|
|R1EUBPUEIXIDP6|          3|           44|         51|   N|                Y|
|R137E1FON7IRZS|          1|           18|         22|   N|                N|
| RE1E7SYQDI27W|          1|           25|         35|   N|     

In [12]:
# Determine total number of reviews
helpful_vine_df.count()

17761

In [13]:
# Determine total number of paid reviews
helpful_vine_y_df.count()

248

In [14]:
# Determine total number of unpaid reviews
helpful_vine_n_df.count()

17513

In [15]:
# Determine number of 5-star reviews
helpful_vine_df.filter(col("star_rating") == 5).count()

5255

In [16]:
# Determine number of 5-star paid reviews
helpful_vine_y_df.filter(col("star_rating") == 5).count()

102

In [17]:
# Determine number of 5-star unpaid reviews
helpful_vine_n_df.filter(col("star_rating") == 5).count()

5153

In [20]:
# Determine percentage of 5-star paid reviews
a = helpful_vine_y_df.count()
b = helpful_vine_y_df.filter(col("star_rating") == 5).count()
percantage_paid = b/a * 100
print(round(percantage_paid,2))

41.13


In [21]:
# Determine percentage of 5-star unpaid reviews
c = helpful_vine_n_df.count()
d = helpful_vine_n_df.filter(col("star_rating") == 5).count()
percentage_unpaid = d/c * 100
print(round(percentage_unpaid,2))

29.42
